In [243]:
import pandas as pd
import seaborn as sns
import json
from datetime import datetime
import requests

In [2]:
def flatten_dict(dictionary, separator='_'):
    items = []
    stack = [((), dictionary)]

    while stack:
        parent_key, current_dict = stack.pop()
        for key, value in current_dict.items():
            new_key = f"{parent_key}{separator}{key}" if parent_key else key
            if isinstance(value, dict):
                stack.append((new_key, value))
            else:
                items.append((new_key, value))

    return dict(items)

In [254]:
usernames = ['taco.jaco', 'f_society5591']
responses = []
for username in usernames:
    url = f"https://www.instagram.com/api/v1/users/web_profile_info/?username={username}"

    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:102.0) Gecko/20100101 Firefox/102.0",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.5",
        "X-CSRFToken": "0cTLhMNY4YSjtCEm0p1OHsEQ5QBbgIUY",
        "X-IG-App-ID": "936619743392459",
        "X-ASBD-ID": "129477",
        "X-IG-WWW-Claim": "0",
        "X-Requested-With": "XMLHttpRequest",
        "Alt-Used": "www.instagram.com",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Sec-GPC": "1"
    }
    
    referrer = "https://www.instagram.com/api/v1/users/web_profile_info/?username=taco.jaco"
    
    response = requests.get(url, headers=headers)
    responses.append(response.json())

In [271]:
flatten_dicts = []

for idx, username_dict in enumerate(responses):
    flatten_dicts.append(username_dict['data']['user'])
    fact_profile = pd.DataFrame.from_dict(flatten_dicts, orient='columns')

In [272]:
fact_profile

,biography,bio_links,fb_profile_biolink,biography_with_entities,blocked_by_viewer,restricted_by_viewer,country_block,eimu_id,external_url,external_url_linkshimmed,...,username,connected_fb_page,pronouns,edge_felix_video_timeline,edge_owner_to_timeline_media,edge_saved_media,edge_media_collections,edge_related_profiles,bio_links_blog,bio_links_
0,👋 Nice to meet you\n🇲🇾 Amateur photographer\n📷...,"[{'title': 'Blog', 'lynx_url': 'https://l.inst...",None,{'raw_text': '👋 Nice to meet you 🇲🇾 Amateur ph...,False,None,False,17844533627831696,https://tacojaco.me/,https://l.instagram.com/?u=https%3A%2F%2Ftacoj...,...,taco.jaco,None,"[he, him]","{'count': 0, 'page_info': {'has_next_page': Fa...","{'count': 97, 'page_info': {'has_next_page': T...","{'count': 0, 'page_info': {'has_next_page': Fa...","{'count': 0, 'page_info': {'has_next_page': Fa...",{'edges': []},https://tacojaco.me,NaN
1,"I like both coding and photographing, some ret...","[{'title': '', 'lynx_url': 'https://l.instagra...",None,{'raw_text': 'I like both coding and photograp...,False,None,False,17847306332550621,https://bio.chanakancloud.net/,https://l.instagram.com/?u=https%3A%2F%2Fbio.c...,...,f_society5591,None,[],"{'count': 1, 'page_info': {'has_next_page': Fa...","{'count': 77, 'page_info': {'has_next_page': T...","{'count': 0, 'page_info': {'has_next_page': Fa...","{'count': 0, 'page_info': {'has_next_page': Fa...",{'edges': []},NaN,https://bio.chanakancloud.net


In [268]:
posts_df = pd.DataFrame()

for flatten_dict in flatten_dicts:
    for link in flatten_dict['bio_links']:
        flatten_dict['bio_links_' + link['title'].lower()] = link['url']
        flatten_dict['pronouns'] = '/'.join(flatten_dict['pronouns']) if isinstance(flatten_dict['pronouns'], list) else flatten_dict['pronouns']

        for node in flattened_dict['edge_owner_to_timeline_media_edges']:
            node = node['node']
            node_type = node['__typename']
            
            if node_type == 'GraphImage':
                dimension_width = node['dimensions']['width']
                dimension_height = node['dimensions']['height']
                
                display_url = node['display_url']
                owner_id = node['owner']['id']
                owner_username = node['owner']['username']
                is_video = node['is_video']
        
                edge = node['edge_media_to_caption']['edges'][0]
                captions = edge['node']['text']
        
                comments_count = node['edge_media_to_comment']['count']
                comments_disabled = node['comments_disabled']
                taken_at = datetime.fromtimestamp(node['taken_at_timestamp'])
                
                like_count = node['edge_liked_by']['count']
        
                taken_location_root = node['location']
                if taken_location_root:
                    taken_location = taken_location_root['name']
        
                graph_image = {
                    'owner_id': owner_id,
                    'owner_username': owner_username,
                    'dimension_width': dimension_width,
                    'dimension_height': dimension_height,
                    'display_url': display_url,
                    'is_video': is_video,
                    'captions': captions,
                    'comments_count': comments_count,
                    'comments_disabled': comments_disabled,
                    'taken_at': taken_at,
                    'like_count': like_count,
                    'taken_location': taken_location
                }
        
            
                gr_image = pd.DataFrame([graph_image])
                posts_df = pd.concat([posts_df, gr_image], ignore_index=True)

In [234]:
posts_df = pd.DataFrame()

for node in flattened_dict['edge_owner_to_timeline_media_edges']:
    node = node['node']
    node_type = node['__typename']
    
    if node_type == 'GraphImage':
        dimension_width = node['dimensions']['width']
        dimension_height = node['dimensions']['height']
        
        display_url = node['display_url']
        owner_id = node['owner']['id']
        owner_username = node['owner']['username']
        is_video = node['is_video']

        edge = node['edge_media_to_caption']['edges'][0]
        captions = edge['node']['text']

        comments_count = node['edge_media_to_comment']['count']
        comments_disabled = node['comments_disabled']
        taken_at = datetime.fromtimestamp(node['taken_at_timestamp'])
        
        like_count = node['edge_liked_by']['count']

        taken_location_root = node['location']
        if taken_location_root:
            taken_location = taken_location_root['name']

        graph_image = {
            'dimension_width': dimension_width,
            'dimension_height': dimension_height,
            'display_url': display_url,
            'owner_id': owner_id,
            'owner_username': owner_username,
            'is_video': is_video,
            'captions': captions,
            'comments_count': comments_count,
            'comments_disabled': comments_disabled,
            'taken_at': taken_at,
            'like_count': like_count,
            'taken_location': taken_location
        }

    
        gr_image = pd.DataFrame([graph_image])
        posts_df = pd.concat([posts_df, gr_image], ignore_index=True)



In [241]:
fact_profile = fact_profile.drop(columns=['edge_owner_to_timeline_media_edges', 'bio_links', 'eimu_id', 'external_url_linkshimmed', 'fbid'], errors='ignore')

In [242]:
fact_profile

,biography,fb_profile_biolink,blocked_by_viewer,restricted_by_viewer,country_block,external_url,followed_by_viewer,follows_viewer,full_name,group_metadata,...,edge_felix_video_timeline_count,edge_felix_video_timeline_edges,edge_felix_video_timeline_page_info_has_next_page,edge_felix_video_timeline_page_info_end_cursor,edge_mutual_followed_by_count,edge_mutual_followed_by_edges,edge_follow_count,edge_followed_by_count,biography_with_entities_raw_text,biography_with_entities_entities
0,👋 Nice to meet you\n🇲🇾 Amateur photographer\n📷...,None,False,None,False,https://tacojaco.me/,False,False,Jovi,None,...,0,[],False,None,0,[],21,57,👋 Nice to meet you\n🇲🇾 Amateur photographer\n📷...,[]


In [238]:
posts_df.dtypes

dimension_width               int64
dimension_height              int64
display_url                  object
owner_id                     object
owner_username               object
is_video                       bool
captions                     object
comments_count                int64
comments_disabled              bool
taken_at             datetime64[ns]
like_count                    int64
taken_location               object
dtype: object

In [189]:
posts_df.to_csv('taco_jaco_posts.csv', index=None)

In [113]:
flattened_dict['pronouns']

'he/him'

In [94]:
flattened_dict['bio_links_blog']

'https://tacojaco.me'